In [86]:
%load_ext autoreload
%autoreload 2

## Loss landscape plot

plot_surface.py

In [ ]:
!python plot_surface.py --model WideResNet --x=-1:1:3 --y=-1:1:3 --model_file model_acc_state.pt --dir_type weights \
--xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --percentage=0.05 --loss_name smooth_crossentropy

## Eigenvalues of hessian

plot_hessian_eigen.py

In [1]:
# !python plot_hessian_eigen.py --model WideResNet --x=-1:1:3 --y=-1:1:3 --model_file model_acc_state.pt --dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn --plot --percentage=0.05 --loss_name smooth_crossentropy

Using pyhessian

In [107]:
import numpy as np
import torch 
from torchvision import datasets, transforms
# from pytorchcv.model_provider import get_model as ptcv_get_model # model

import matplotlib.pyplot as plt

In [108]:
from my_pyhessian import hessian, utils# Hessian computation
from my_pyhessian.density_plot import get_esd_plot

In [109]:
get_esd_plot

<function my_pyhessian.density_plot.get_esd_plot(eigenvalues, weights, name='ex.png')>

In [118]:
stored = torch.load('../to_plot/model_cifar_SGD.pt', map_location=lambda storage, loc: storage)
model = WideResNet(8, 2, 0.0, in_channels=3, labels=10)

if 'state_dict' in stored.keys():
    model.load_state_dict(stored['state_dict'])
else:
    model.load_state_dict(stored)
model.eval()

# create loss function
# criterion = torch.nn.CrossEntropyLoss()
criterion = mean_smooth_crossentropy

# get dataset 
dataset = Cifar(0.3, 128, 2)
trainloader, testloader = dataset.train, dataset.test


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [119]:
# for illustrate, we only use one batch
for inputs, targets in trainloader:
    break

# For multiple batches
dataloader = trainloader

# we use cuda to make the computation fast
# model = model.cuda()
# inputs, targets = inputs.cuda(), targets.cuda()

In [120]:
# create the hessian computation module
hessian_comp = hessian(model, criterion, data=(inputs, targets), cuda=False) # Single batch
# hessian_comp = hessian(model, criterion, dataloader=dataloader, cuda=False) # Whole training set

In [125]:
density_eigen, density_weight = hessian_comp.density(iter=100, n_v=1)

In [126]:
density_eigen

[[(229.96605+0j),
  (193.21214+0j),
  (186.26375+0j),
  (162.7291+0j),
  (135.69576+0j),
  (118.33687+0j),
  (101.612816+0j),
  (82.73761+0j),
  (57.73179+0j),
  (34.82363+0j),
  (33.903225+0j),
  (26.515314+0j),
  (24.743633+0j),
  (22.087645+0j),
  (19.203455+0j),
  (18.300983+0j),
  (17.002703+0j),
  (16.334206+0j),
  (16.188618+0j),
  (15.047378+0j),
  (14.6554+0j),
  (13.519853+0j),
  (12.6556835+0j),
  (12.233987+0j),
  (11.745465+0j),
  (11.332093+0j),
  (10.910723+0j),
  (10.854833+0j),
  (10.500712+0j),
  (9.7851095+0j),
  (-6.198369+0j),
  (9.384801+0j),
  (8.777609+0j),
  (8.610484+0j),
  (8.407791+0j),
  (8.0701885+0j),
  (8.249808+0j),
  (-5.8806424+0j),
  (7.7146363+0j),
  (7.1189804+0j),
  (7.436264+0j),
  (7.3544536+0j),
  (-5.685235+0j),
  (-5.5582166+0j),
  (-5.1588693+0j),
  (-5.00667+0j),
  (6.7694507+0j),
  (6.606938+0j),
  (6.4528723+0j),
  (-4.8453946+0j),
  (-4.69743+0j),
  (6.1706343+0j),
  (5.894369+0j),
  (-4.642734+0j),
  (5.237677+0j),
  (4.9562664+0j),
  (

In [128]:
get_esd_plot(density_eigen, density_weight,'eig_cifar_sgd.png')

C:\Users\franc\Desktop\Francesco\EPFL_Courses\Optimization for ML\sharpness-aware_minimization\loss_landscape\my_pyhessian\density_plot.py:64: ComplexWarning: Casting complex values to real discards the imaginary part
  normalization = np.sum(density) * (grids[1] - grids[0])


In [ ]:
# Now let's compute the top eigenvalue.
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(maxIter=10, top_n = 1)
print("The top Hessian eigenvalue of this model is %.4f"%top_eigenvalues[-1])